<h2 align="center">Machine Learning</h2> 

<h3 align="center"> Clustering and Principal Component Analysis</h3> 


# Outline 
      
1. Preprocessing Intro 
2. PCA 
3. Clustering Concepts
4. K Means Clustering

### Chapter 2 Content to Understand before we dive in:
* Regression vs Classification
* K-NN, decision boundary
* Linear Models
    - OLS, Ridge, Lasso, LogisticRegression
* Naive Bayes
* Decision Trees
    - Feature Importance
* Ensembles, Random Forest
* Support Vector Machines (SVM)
* Neural Networks

### That's a lot.  We will revisit these topics in detail.

### Unsupervised learning includes all kinds of machine learning where there is no known output when training.


Why do we do this?
1. Visualization
2. Data Compression
3. Improvement: Making the data a better input for a supervised model.

# From the top: It may be difficult to evaluate whether an unsupervised model has a powerful prediction

It is hard to determine if the model has done WELL or not, because we do not have any "actuals"

# Unsupervised models are frequently used in a "tell me more about the data" stage: exploration

# Preprocessing Methods

Chart First (from Guido + Muller's Introduction to Machine Learning with Python)

<center><img src="../images/scaling_example.jpg" alt="drawing" style="width: 1000px;"/></center>


### Why Preprocessing ???

Simple.  We want to increase the efficacy of our predictions.

Recall the Mastercard bounty example from Class 1.

# We see Four Different Scalers  

1. Standard Scaler
    * Mean is 0, variance is 1
2. Robust Scaler
    * Similar to Standard Scaler, but uses median and quartiles as opposed to mean and variance
    * Result: Resistant to outliers, just like robust regressors etc
3. Min/Max Scaler
    * Shifts the feature(s) such that all data is between 0 and 1
4. Normalizer
    * Scales data such that features have Euclidean length of 1

# Sklearn has some great visualizations on this:
https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

They have also made a notebook file available at that link.  Let's check it out!

In [2]:
import pandas
import postgresql
import pickle
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
# Grab the data, put in DataFrame
# This is the US Census data on incomes
df = pandas.read_pickle('C:\\Users\\Travis Millburn\\OneDrive - University of New Haven\\Class Materials\\Spring 2024\\Week 4\\df.pkl')

# Apply our encoding magic!
for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
        
x_df = df.drop(columns=['income', 'fnlwgt'])
y_df = df['income']
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df)
        
# What does that give us?
df.tail()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
32556,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0,32557
32557,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1,32558
32558,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0,32559
32559,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0,32560
32560,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1,32561


In [4]:
# 1 Neighbor, no preprocessing

clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.73


In [5]:
# 5 Neighbors, no preprocessing

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.78


### Okay, that gets us back to where we were before.  What does preprocessing do?

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [7]:
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.79


In [8]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.82


### What about RobustScaler ?

In [9]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_df_scaled = scaler.fit_transform(x_df)
x_train, x_test, y_train, y_test = train_test_split(x_df_scaled, y_df)

In [10]:
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_train, y_train)
print('Accuracy: {:.2f}'.format(clf.score(x_test, y_test)))

Accuracy: 0.85


### We got several percentage points of improvement by scaling the data prior to fitting the K-NN model.

# At this point we have introducted scaling......we will later integrate with some of supervised models we have seen.

### Time for Lab A !

# PCA: Principal Component Analysis

One of the most prevalent unsupervised algorithms is PCA.

Dimensionality Reduction.

Methodology rotates data such that rotated features on uncorrelated.

This can be done for accuracy OR performance  
   * Oftentimes, running data through PCA and then using it as input for a supervised model can be good for performance


<center><img src="../images/pca.jpg" alt="drawing" style="width: 1000px;"/></center>

Introduction to Machine Learning; Guido and Muller

# Example: Let's (further) explore the popular wine dataset

In [11]:
import pandas
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
%matplotlib inline


In [12]:
#train_test_split from last class

# X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=1)
red_df = pandas.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', delimiter=';')
red_df.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


### PCA via Eigenvector decomposition

* Data matrix $\bf X$ - _rows are features_

* Consider matrix $\boldsymbol\Sigma = \bf X \bf X^T$ - covariance estimate  - each element is a covariance between pairs of features (note need to remove mean in definition... and in data).

* eigenvalue decomposition gives directions of variation

In [13]:
red_x_train, red_x_test, red_y_train, red_y_test = train_test_split(red_df.drop(columns=['quality']), red_df['quality'])

In [14]:
pca = PCA(n_components=2)
pca.fit(red_x_train)

PCA(n_components=2)

In [15]:
# x_pca

In [16]:
x_pca = pca.transform(red_x_train)
x_pca_test = pca.transform(red_x_test)

In [17]:
print("Original shape: {}".format(str(red_x_train.shape)))
print("Reduced shape: {}".format(str(x_pca.shape)))

Original shape: (1199, 11)
Reduced shape: (1199, 2)


In [45]:
clf = KNeighborsRegressor()
clf.fit(x_pca, red_y_train)

KNeighborsRegressor()

In [46]:
red_y_results = pandas.DataFrame(red_y_test)
red_y_results['prediction'] = clf.predict(x_pca_test)
red_y_results['pred_rounded'] = red_y_results['prediction'].round().astype(int)
red_y_results['error'] = red_y_results['pred_rounded'] - red_y_results['quality']

In [47]:
red_y_results['error'].abs().mean()

# How does this compare to just a KNN model on the same data?|

0.5825

In [48]:
red_y_results.tail()

,quality,prediction,pred_rounded,error
1003,7,5.6,6,-1
1056,7,5.8,6,-1
1258,6,6.2,6,0
1537,6,5.2,5,-1
364,7,5.8,6,-1


# Now, to some Clustering

K-Means Clustering is likely the most commonly used clustering algorithm

We are going to try and classify data, without first being trained!
Why would we do this?
    * Search engines
    * Customer profiling

In [49]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(red_x_train)

C:\Program Files\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3)

In [50]:
print("Cluster memberships:\n{}".format(kmeans.labels_))

Cluster memberships:
[2 1 2 ... 2 1 1]
